### **TensorFlow Keras Code**

In [ ]:
# !unzip /content/drive/MyDrive/apple_vs_banana/apples-bananas-oranges.zip -d /content

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define input shape and number of classes
img_shape = (224, 224, 3)
num_classes = 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Set up data directories
train_dir = '/content/drive/MyDrive/apple_banana/data/train'
test_dir = '/content/drive/MyDrive/apple_banana/data/test'

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary',
        subset='validation')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary',
        shuffle=False)

Found 289 images belonging to 2 classes.
Found 71 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose, Flatten, Dense
from tensorflow.keras.applications import InceptionV3

# Define input shape and number of classes
input_shape = (299, 299, 3)
# Define InceptionV3 backbone
backbone = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')

# Set the backbone layers to be non-trainable
for layer in backbone.layers:
    layer.trainable = False

# Define U-Net architecture with InceptionV3 backbone
inputs = Input(input_shape)
x1 = backbone(inputs)

# Downsampling
conv1a = Conv2D(32, (3, 3), activation='relu', padding='same')(x1)
conv1b = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1a)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1b)

conv2a = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2b = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2a)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2b)

conv3a = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2b)
conv3b = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3a)

# Upsampling
up4 = Conv2DTranspose(64, (2, 2), strides=(1, 1), padding='same')(conv3b)
merge4 = concatenate([up4, conv2b], axis=3)
conv4a = Conv2D(64, (3, 3), activation='relu', padding='same')(merge4)
conv4b = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4a)

up5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv4b)
merge5 = concatenate([up5, conv1b], axis=3)
conv5a = Conv2D(32, (3, 3), activation='relu', padding='same')(merge5)
conv5b = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5a)

x2 = Flatten()(conv5b)
x3 = Dense(512, activation='relu')(x2)
x3 = Dropout(0.5)(x3)
x4 = Dense(256, activation='relu')(x3)
x4 = Dropout(0.2)(x4)
x5 = Dense(64, activation='relu')(x4)
x5 = Dropout(0.2)(x5)
outputs = Dense(1, activation='sigmoid')(x5)

# Define the model
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)


# # Compile the model with categorical crossentropy loss and Adam optimizer
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print the model summary
model.summary()


87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 inception_v3 (Functional)      (None, 8, 8, 2048)   21802784    ['input_2[0][0]']                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 8, 8, 32)     589856      ['inception_v3[0][0]']           
                                                                                                  
 conv2d_95 (Conv2D)          

In [ ]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)

# Evaluate model

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)


Epoch 1/10
9/9 [==============================] - 137s 14s/step - loss: 0.3479 - accuracy: 0.7860 - val_loss: 9.4924e-04 - val_accuracy: 1.0000
Epoch 2/10
9/9 [==============================] - 95s 11s/step - loss: 8.3559e-04 - accuracy: 1.0000 - val_loss: 0.0355 - val_accuracy: 0.9859
Epoch 3/10
9/9 [==============================] - 91s 10s/step - loss: 1.5190e-04 - accuracy: 1.0000 - val_loss: 1.7884e-07 - val_accuracy: 1.0000
Epoch 4/10
9/9 [==============================] - 92s 10s/step - loss: 5.6210e-09 - accuracy: 1.0000 - val_loss: 6.3188e-12 - val_accuracy: 1.0000
Epoch 5/10
9/9 [==============================] - 92s 11s/step - loss: 1.0304e-11 - accuracy: 1.0000 - val_loss: 5.6402e-14 - val_accuracy: 1.0000
Epoch 6/10
9/9 [==============================] - 101s 12s/step - loss: 1.8457e-13 - accuracy: 1.0000 - val_loss: 6.4208e-15 - val_accuracy: 1.0000
Epoch 7/10
9/9 [==============================] - 109s 13s/step - loss: 1.6453e-12 - accuracy: 1.0000 - val_loss: 2.5278e-15

In [ ]:
model.save('content/apple_vs_banana_unet_keras(v.1).h5')

In [ ]:
# Evaluate model on test set
loss, accuracy = model.evaluate(test_generator)

# Print test set accuracy
print("Test set accuracy: {:.2f}%".format(accuracy * 100))

3/3 [==============================] - 28s 9s/step - loss: 1.0923e-16 - accuracy: 1.0000
Test set accuracy: 100.00%


In [ ]:
# Generate predictions for test set
predictions = model.predict(test_generator)

# Print predicted class probabilities for the first image in the test set
print("Predicted class probabilities:", predictions[0])


3/3 [==============================] - 33s 10s/step
Predicted class probabilities: [0.]


In [ ]:
# Load the model from the file
from tensorflow.keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/apple_vs_banana_unet_keras(v.1).h5')


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Load the new image
img = load_img('/content/Untitled.png', target_size=(299,299,3))

# Preprocess the image
img_array = img_to_array(img)
img_array = img_array[np.newaxis, ...] / 255.

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the predicted class label
class_labels = train_generator.class_indices
predicted_label = int(np.round(predictions))
predicted_class = list(class_labels.keys())[list(class_labels.values()).index(predicted_label)]

# Print the predicted class
print("Predicted class:", predicted_class)

1/1 [==============================] - 2s 2s/step
Predicted class: apple


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Generate predictions for test set
predictions = loaded_model.predict(test_generator)

# Get true labels for test set
true_labels = test_generator.classes

# Convert predicted labels to class indices
predicted_labels = np.round(predictions)

# Generate confusion matrix and classification report
confusion = confusion_matrix(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=list(test_generator.class_indices.keys()))

# Print confusion matrix and classification report
print("Confusion Matrix:\n", confusion)
print("\nClassification Report:\n", report)

3/3 [==============================] - 1s 231ms/step
Confusion Matrix:
 [[47  0]
 [ 0 44]]

Classification Report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        47
      banana       1.00      1.00      1.00        44

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91

